<a href="https://colab.research.google.com/github/weedge/doraemon-nb/blob/main/daily_ocr_vision_bot.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [58]:
!lscpu && nvidia-smi

Architecture:             x86_64
  CPU op-mode(s):         32-bit, 64-bit
  Address sizes:          46 bits physical, 48 bits virtual
  Byte Order:             Little Endian
CPU(s):                   2
  On-line CPU(s) list:    0,1
Vendor ID:                GenuineIntel
  Model name:             Intel(R) Xeon(R) CPU @ 2.00GHz
    CPU family:           6
    Model:                85
    Thread(s) per core:   2
    Core(s) per socket:   1
    Socket(s):            1
    Stepping:             3
    BogoMIPS:             4000.32
    Flags:                fpu vme de pse tsc msr pae mce cx8 apic sep mtrr pge mca cmov pat pse36 cl
                          flush mmx fxsr sse sse2 ss ht syscall nx pdpe1gb rdtscp lm constant_tsc re
                          p_good nopl xtopology nonstop_tsc cpuid tsc_known_freq pni pclmulqdq ssse3
                           fma cx16 pcid sse4_1 sse4_2 x2apic movbe popcnt aes xsave avx f16c rdrand
                           hypervisor lahf_lm abm 3dnowprefetch i

In [ ]:
!cd /content && rm -rf achatbot && git clone --recursive https://github.com/ai-bot-pro/achatbot.git

In [ ]:
%cd /content/achatbot

In [ ]:
!bash scripts/pypi_achatbot.sh dev

In [ ]:
!pip install -q "dist/achatbot-0.0.7.2-py3-none-any.whl[daily_room_audio_stream,sense_voice_asr,vision_transformers_got_ocr,tts_edge,queue,flash-attn,tensorrt]" --no-build-isolation

In [ ]:
!pip show transformers flash-attn tensorrt torch torchaudio torchvision

In [ ]:
!huggingface-cli download FunAudioLLM/SenseVoiceSmall  --local-dir ./models/FunAudioLLM/SenseVoiceSmall --local-dir-use-symlinks False

In [ ]:
!huggingface-cli download stepfun-ai/GOT-OCR2_0 --local-dir ./models/stepfun-ai/GOT-OCR2_0 --local-dir-use-symlinks False

In [ ]:
# cpu bfloat16
!LOG_LEVEL=debug LLM_MODEL_NAME_OR_PATH=./models/stepfun-ai/GOT-OCR2_0 \
    python -m unittest test.modules.vision.ocr.test_transformers_got.TestTransformersGOTOCR.test_ocr_generate

In [ ]:
# gpu float16
!LOG_LEVEL=debug LM_DEVICE=cuda LM_TORCH_DTYPE=float16 LLM_MODEL_NAME_OR_PATH=./models/stepfun-ai/GOT-OCR2_0 \
    python -m unittest test.modules.vision.ocr.test_transformers_got.TestTransformersGOTOCR.test_ocr_generate

In [ ]:
# cpu bfloat16
!LOG_LEVEL=debug python -m unittest test.integration.processors.test_vision_ocr_processor.TestProcessor.test_run

In [ ]:
# gpu float16
!LOG_LEVEL=debug LM_DEVICE=cuda LM_TORCH_DTYPE=float16 python -m unittest test.integration.processors.test_vision_ocr_processor.TestProcessor.test_run

In [ ]:
from google.colab import userdata
DAILY_API_KEY=userdata.get('DAILY_API_KEY')


run bot task woker with bot.json, e.g.: daily_describe_transformers_vision_bot.json

use daily room stream, u can click bot joined the room url e.g.: https://weedge.daily.co/chat-bot to start chat with bot with audio and camera stream




In [55]:
!cat /content/daily_ocr_vision_bot.json

{
  "chat_bot_name": "DailyOCRVisionBot",
  "room_name": "chat-room",
  "room_url": "",
  "token": "",
  "services": {},
  "config": {
    "vad": {
      "tag": "silero_vad_analyzer",
      "args": { "stop_secs": 0.7 }
    },
    "asr": {
      "tag": "sense_voice_asr",
      "args": {
        "language": "zn",
        "model_name_or_path": "./models/FunAudioLLM/SenseVoiceSmall"
      }
    },
    "vision_ocr": {
      "tag": "vision_transformers_got_ocr",
      "args": {
        "lm_model_name_or_path": "./models/stepfun-ai/GOT-OCR2_0",
        "lm_torch_dtype": "float16",
        "model_type": "chat",
        "ocr_type": "ocr",
        "ocr_box": "",
        "ocr_color": "",
        "lm_device": "cuda"
      }
    },
    "tts": {
      "tag": "tts_edge",
      "args": {
        "voice_name": "zh-CN-YunjianNeural",
        "language": "zh",
        "gender": "Male"
      }
    }
  },
  "config_list": []
}


In [ ]:
!DAILY_API_KEY=$DAILY_API_KEY python -m src.cmd.bots.main -f /content/daily_ocr_vision_bot.json

In [ ]:
from google.colab import userdata
REDIS_PASSWORD=userdata.get('REDIS_PASSWORD')

In [ ]:
!LOG_LEVEL=debug REDIS_PASSWORD=$REDIS_PASSWORD python -m src.cmd.bots.main -f /content/task_bot.json